In [2]:
import pandas as pd

from datetime import timedelta, date
from quantbullet.utils.data import generate_fake_bond_trades
from quantbullet.model_selection.rolling_window import RollingWindowManager, RollingWindowBacktester
from quantbullet.model import Model, Feature, FeatureSpec, FeatureRole, DataType

In [3]:
%load_ext autoreload
%autoreload 2

## Generate Synthetic Data

In [4]:
START_DATE = date( 2025, 1, 1 )
END_DATE = date( 2025, 4, 30 )

In [5]:
df = generate_fake_bond_trades( start_date=START_DATE, end_date=END_DATE )

In [6]:
df.head()

,date,ticker,rating,feature_A,feature_B,feature_C,feature_D,expiry,yield
0,2025-01-01,TICK004 AAA,AAA,1.251888,-0.365574,0.449846,1.084808,2031-07-26,5.429837
1,2025-01-01,TICK038 AAA,AAA,-0.020429,1.186029,0.074927,-0.060175,2033-11-25,6.562629
2,2025-01-01,TICK037 AAA,AAA,0.784997,-0.558714,1.355466,-1.959671,2034-02-22,4.991440
3,2025-01-01,TICK023 AAA,AAA,-2.352641,0.483961,0.424343,0.444225,2026-12-01,3.102583
4,2025-01-01,TICK034 AAA,AAA,-0.928687,-1.469315,1.637445,0.136618,2029-06-07,1.136793


## Define Model and Features

In [7]:
AverageYieldModelFeatureSpec = FeatureSpec(
    [
        Feature( "rating", DataType.CATEGORICAL, FeatureRole.MODEL_INPUT),
        Feature( "yield", DataType.FLOAT, FeatureRole.TARGET)
    ]
)

In [8]:
class AverageYieldModel( Model ):
    def __init__( self, feature_spec: FeatureSpec ):
        super().__init__( feature_spec )

    def fit( self, X, y=None ):
        self._rating_map = X.groupby( "rating" ).agg( { "yield": "mean" } ).reset_index()
        self._rating_map = self._rating_map.set_index( "rating" ).to_dict()

    def predict( self, X ):
        return X[ "rating" ].map( self._rating_map[ "yield" ] ).values

In [9]:
rwm = RollingWindowManager.from_flat( df, window_size=30 )
model = AverageYieldModel( feature_spec=AverageYieldModelFeatureSpec )
backtester = RollingWindowBacktester( rolling_window_manager=rwm, model=model )

In [10]:
backtester.run( use_multithreading=False )

In [11]:
backtester.get_logger_df().head()

,date,ticker,rating,feature_A,feature_B,feature_C,feature_D,expiry,yield,loggerPred,loggerRunDate,loggerResid
0,2025-01-31,TICK008 BBB,BBB,-0.512888,-0.311519,-0.025630,1.382923,2025-05-29,3.804156,5.053221,2025-01-30,-1.249065
1,2025-01-31,TICK043 BBB,BBB,0.293230,0.751977,-0.299135,0.071018,2028-07-16,5.616794,5.053221,2025-01-30,0.563573
2,2025-01-31,TICK017 BBB,BBB,0.063136,0.369896,-0.926765,0.041201,2029-02-11,5.866545,5.053221,2025-01-30,0.813324
3,2025-01-31,TICK007 BBB,BBB,-0.279497,0.178586,1.218134,-0.756868,2034-07-01,4.966024,5.053221,2025-01-30,-0.087198
4,2025-01-31,TICK042 BBB,BBB,-0.231019,1.940160,1.277303,0.231858,2032-11-09,9.224095,5.053221,2025-01-30,4.170874
